In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import xgboost as xgb

In [ ]:
from google.colab import files

print(files.upload())

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c convolve2-epoch-1

Saving kaggle.json to kaggle.json
{'kaggle.json': b'{"username":"adityatamras","key":"85291e7c2b635b7a48a56e2a5b01fe0e"}'}
100% 194k/194k [00:00<00:00, 718kB/s]
100% 194k/194k [00:00<00:00, 715kB/s]


In [ ]:
!unzip /content/convolve2-epoch-1.zip

Archive:  /content/convolve2-epoch-1.zip
  inflating: Dataset.csv             
  inflating: sample_submission - Sheet1 (1).csv  


In [ ]:
data = pd.read_csv("/content/Dataset.csv")

In [ ]:
print("Number of columns with 'y' = 1:", (data['y'] == 1).sum())

Number of columns with 'y' = 1: 451


In [ ]:
print("Number of columns with 'y' = 0:", (data['y'] == 0).sum())

Number of columns with 'y' = 0: 3668


In [ ]:
data.describe()

,Unnamed: 0,x_1,x_109,x_113,x_12,x_124,x_127,x_144,x_16,x_18,...,x_73,x_77,x_78,x_8,x_80,x_83,x_89,x_91,x_99,y
count,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,...,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000
mean,2059.000000,5166.469289,0.563244,3.048798,503.509832,50.483855,-0.122845,3.609128,34.826657,960.422190,...,-0.032340,1050.474387,16.507405,256.788055,-0.133831,248.378733,0.000371,-0.006766,-0.037985,0.109493
std,1189.197208,73.588512,0.496044,2.544466,291.454491,28.862659,1.468947,1.825079,20.441569,191.922786,...,0.289226,408.389794,9.554037,254.703736,0.572124,145.247366,0.044075,0.156240,0.380439,0.312294
min,0.000000,4964.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,1.000000,0.000000,0.000000,...,-0.620000,49.000000,0.000000,0.000000,-0.310000,0.000000,-0.180000,-0.130000,-0.660000,0.000000
25%,1029.500000,5099.000000,0.000000,1.000000,250.000000,26.000000,-2.000000,1.000000,17.000000,999.000000,...,0.110000,766.500000,8.000000,103.000000,-0.310000,122.000000,0.020000,-0.100000,-0.490000,0.000000
50%,2059.000000,5191.000000,1.000000,5.000000,497.000000,50.000000,1.000000,5.000000,35.000000,999.000000,...,0.110000,1048.000000,16.000000,181.000000,-0.310000,249.000000,0.020000,-0.100000,0.070000,0.000000
75%,3088.500000,5228.000000,1.000000,6.000000,760.500000,76.000000,1.000000,5.000000,53.000000,999.000000,...,0.110000,1332.500000,25.000000,317.000000,-0.310000,374.000000,0.020000,0.240000,0.210000,0.000000
max,4118.000000,5228.000000,1.000000,6.000000,999.000000,100.000000,1.000000,5.000000,70.000000,999.000000,...,0.110000,2077.000000,33.000000,3643.000000,2.690000,500.000000,0.020000,0.390000,0.930000,1.000000


In [ ]:
data.head()

,Unnamed: 0,x_1,x_109,x_113,x_12,x_124,x_127,x_144,x_16,x_18,...,x_73,x_77,x_78,x_8,x_80,x_83,x_89,x_91,x_99,y
0,0,5099,1,5,156,35,-2,1,40,999,...,0.11,629,14,487,-0.31,115,0.02,-0.10,-0.49,0
1,1,5191,0,1,164,16,1,5,4,999,...,0.11,509,26,346,-0.31,317,0.02,0.24,-0.21,0
2,2,5228,1,0,774,75,1,5,12,999,...,0.11,1058,30,227,-0.31,464,0.02,-0.10,-0.21,0
3,3,5228,1,1,747,4,1,5,69,999,...,0.11,1048,4,17,-0.31,39,-0.18,-0.10,-0.21,0
4,4,5196,1,5,29,28,0,4,32,999,...,0.11,993,22,58,-0.31,492,0.02,-0.10,0.21,0


In [ ]:
df_1 = data[data['y'] == 1].reset_index(drop = True)

In [ ]:
df_1

,Unnamed: 0,x_1,x_109,x_113,x_12,x_124,x_127,x_144,x_16,x_18,...,x_73,x_77,x_78,x_8,x_80,x_83,x_89,x_91,x_99,y
0,19,5228,1,0,342,24,1,5,3,999,...,0.11,987,26,849,-0.31,445,-0.07,-0.10,-0.49,1
1,21,5009,0,5,403,61,-2,1,66,12,...,0.11,789,29,222,2.69,457,0.02,-0.13,0.07,1
2,25,5228,0,0,971,47,1,5,19,999,...,0.11,752,32,479,-0.31,354,0.02,-0.10,0.07,1
3,53,5018,1,5,508,40,-3,1,64,3,...,0.11,992,32,89,2.69,226,-0.07,-0.10,0.07,1
4,59,4964,0,5,63,37,-1,1,56,999,...,0.11,948,13,481,0.35,313,0.02,-0.13,0.21,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,4034,5076,1,6,380,14,-3,1,1,999,...,0.11,1747,11,232,-0.31,195,-0.07,-0.10,0.90,1
447,4066,4964,1,6,42,37,-1,1,34,999,...,0.11,1189,5,621,0.35,48,0.02,-0.10,0.90,1
448,4069,5099,1,0,97,79,-2,1,40,999,...,0.11,1620,24,1602,-0.31,489,0.02,-0.10,0.21,1
449,4088,5018,0,0,475,43,-3,1,6,3,...,0.11,1347,26,323,2.69,19,0.02,-0.10,-0.16,1


In [ ]:
df_0 = data[data['y'] == 0].reset_index(drop = True)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:

columns_to_drop = ['y', 'Unnamed: 0']
y = np.array(data['y'])
X = data.drop(columns_to_drop, axis=1)


smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)


columns = X.columns
data_resampled = pd.DataFrame(X_resampled, columns=columns)
data_resampled['y'] = y_resampled


print("Original dataset shape:", data.shape)
print("Resampled dataset shape:", data_resampled.shape)


Original dataset shape: (4119, 42)
Resampled dataset shape: (7336, 41)


In [ ]:
data_resampled

,x_1,x_109,x_113,x_12,x_124,x_127,x_144,x_16,x_18,x_2,...,x_73,x_77,x_78,x_8,x_80,x_83,x_89,x_91,x_99,y
0,5099,1,5,156,35,-2,1,40,999,7,...,0.110000,629,14,487,-0.310000,115,0.02,-0.100000,-0.490000,0
1,5191,0,1,164,16,1,5,4,999,5,...,0.110000,509,26,346,-0.310000,317,0.02,0.240000,-0.210000,0
2,5228,1,0,774,75,1,5,12,999,4,...,0.110000,1058,30,227,-0.310000,464,0.02,-0.100000,-0.210000,0
3,5228,1,1,747,4,1,5,69,999,3,...,0.110000,1048,4,17,-0.310000,39,-0.18,-0.100000,-0.210000,0
4,5196,1,5,29,28,0,4,32,999,6,...,0.110000,993,22,58,-0.310000,492,0.02,-0.100000,0.210000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7331,5009,0,0,329,85,-2,1,58,6,1,...,0.110000,1558,20,381,2.690000,269,0.02,-0.095434,-0.155031,1
7332,5220,0,4,670,60,0,4,43,999,5,...,0.110000,704,31,869,-0.273089,11,0.02,0.220985,0.170852,1
7333,4967,1,4,125,41,-1,1,44,5,5,...,0.006208,1436,15,683,2.690000,48,0.02,-0.100000,0.308105,1
7334,5190,0,5,868,61,0,3,37,999,1,...,0.110000,1150,25,1089,-0.275135,114,0.02,-0.100000,-0.054300,1


In [ ]:
print("Number of columns with 'y' = 1:", (data_resampled['y'] == 1). sum())
print("Number of columns with 'y' = 0:", (data_resampled['y'] == 0). sum())

Number of columns with 'y' = 1: 3668
Number of columns with 'y' = 0: 3668


In [ ]:

correlation_matrix = data_resampled.corr().abs()


upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))


threshold = 0.09
high_corr_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]


selected_features_df = data_resampled[high_corr_features]


X_selected = selected_features_df.drop('y', axis=1)
y_selected = selected_features_df['y']



num_features_to_select = 20
skb = SelectKBest(score_func=f_classif, k='all')
X_selected_kbest = skb.fit_transform(X_selected, y_selected)


selected_feature_indices = skb.get_support(indices=True)


selected_feature_names = X_selected.columns[selected_feature_indices]


print("Selected features:")
print(selected_feature_names)

feature_names = list(X_resampled.columns)

feature_y_df = pd.DataFrame({'Variables': feature_names, 'y': 0})

feature_y_df['y'] = feature_y_df['Variables'].apply(lambda x: 1 if x in selected_feature_names else 0)
feature_y_df = feature_y_df.reset_index(drop=True)
feature_y_df = feature_y_df[feature_y_df['Variables'] != 'y']
feature_y_df

In [ ]:
feature_y_df.to_csv('correlation.csv', index=False)

In [ ]:
lgbm_df = pd.read_csv('/content/LGBM.csv')
corr_df = pd.read_csv('/content/correlation.csv')

In [ ]:
lgbm_df = lgbm_df.rename(columns={'Mark': 'y'})
print(lgbm_df)

In [ ]:
merged_df = pd.merge(lgbm_df, corr_df, on='Variables', suffixes=('_df1', '_df2'))
filtered_df = merged_df[((merged_df['y_df1'] == merged_df['y_df2']) & (merged_df['y_df1'] == 0)) | merged_df['y_df1'] != merged_df['y_df2']]
filtered_df = filtered_df[['Variables', 'y_df1']].rename(columns={'y_df1': 'y'})
filtered_df.reset_index(drop=True, inplace=True)

print(filtered_df)


   Variables  y
0        x_1  1
1      x_109  0
2      x_113  0
3       x_12  0
4      x_124  0
5       x_16  0
6        x_2  0
7       x_30  0
8       x_33  1
9       x_42  0
10      x_44  0
11      x_55  0
12      x_59  0
13      x_62  0
14      x_64  0
15      x_67  0
16      x_68  0
17       x_7  0
18      x_70  0
19      x_71  1
20      x_77  0
21      x_78  0
22      x_83  0


In [ ]:
cols_to_keep = list(filtered_df['Variables'])
print(cols_to_keep)
X_resampled = X_resampled[cols_to_keep]

['x_1', 'x_109', 'x_113', 'x_12', 'x_124', 'x_16', 'x_2', 'x_30', 'x_33', 'x_42', 'x_44', 'x_55', 'x_59', 'x_62', 'x_64', 'x_67', 'x_68', 'x_7', 'x_70', 'x_71', 'x_77', 'x_78', 'x_83']


In [ ]:
adaboost = AdaBoostClassifier()
random_forest = RandomForestClassifier()
xgb_classifier = xgb.XGBClassifier()


adaboost.fit(X_resampled, y_resampled)
random_forest.fit(X_resampled, y_resampled)
xgb_classifier.fit(X_resampled, y_resampled)


feature_importances_ada = adaboost.feature_importances_
feature_importances_rf = random_forest.feature_importances_
feature_importances_xgb = xgb_classifier.feature_importances_


feature_importance_df_ada = pd.DataFrame({'Feature': X_resampled.columns, 'Importance_AdaBoost': feature_importances_ada})
feature_importance_df_rf = pd.DataFrame({'Feature': X_resampled.columns, 'Importance_RandomForest': feature_importances_rf})
feature_importance_df_xgb = pd.DataFrame({'Feature': X_resampled.columns, 'Importance_XGBoost': feature_importances_xgb})


feature_importance_combined = pd.merge(feature_importance_df_ada, feature_importance_df_rf, on='Feature')
feature_importance_combined = pd.merge(feature_importance_combined, feature_importance_df_xgb, on='Feature')


feature_importance_combined['Average_Importance'] = feature_importance_combined.mean(axis=1)


top_3_features = feature_importance_combined.sort_values(by='Average_Importance', ascending=False).head(3)
print(top_3_features)


   Feature  Importance_AdaBoost  Importance_RandomForest  Importance_XGBoost  \
0      x_1                 0.10                 0.249761            0.275854   
11    x_55                 0.16                 0.075364            0.179738   
13    x_62                 0.24                 0.063074            0.081456   

    Average_Importance  
0             0.208539  
11            0.138367  
13            0.128176  


<ipython-input-67-9a6a04113e65>:31: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  feature_importance_combined['Average_Importance'] = feature_importance_combined.mean(axis=1)


In [ ]:
filtered_df_2 = merged_df[((merged_df['y_df1'] == merged_df['y_df2']) & (merged_df['y_df1'] == 1))]
filtered_df_2 = filtered_df_2[['Variables', 'y_df1']].rename(columns={'y_df1': 'y'})
filtered_df_2.reset_index(drop=True, inplace=True)
filtered_df_2

,Variables,y
0,x_127,1
1,x_144,1
2,x_18,1
3,x_25,1
4,x_27,1
5,x_28,1
6,x_38,1
7,x_45,1
8,x_52,1
9,x_57,1


In [ ]:
selected_vars = list(filtered_df_2['Variables'])
add = ['x_1', 'x_55', 'x_62']
selected_vars.extend(add)
print(selected_vars)

['x_127', 'x_144', 'x_18', 'x_25', 'x_27', 'x_28', 'x_38', 'x_45', 'x_52', 'x_57', 'x_60', 'x_73', 'x_8', 'x_80', 'x_89', 'x_91', 'x_99', 'x_1', 'x_55', 'x_62']


In [ ]:
new_df = pd.DataFrame({'Variables': feature_names, 'Mark': 0})
new_df['y'] = new_df['Variables'].apply(lambda x: 1 if x in selected_vars else 0)
new_df = new_df[new_df['Variables'] != 'y']
new_df = new_df.reset_index(drop=True)
new_df

In [ ]:
new_df.to_csv('final_submission.csv', index=False)